In [ ]:

######## snakemake preamble start (automatically inserted, do not edit) ########
library(methods)
Snakemake <- setClass(
    "Snakemake",
    slots = c(
        input = "list",
        output = "list",
        params = "list",
        wildcards = "list",
        threads = "numeric",
        log = "list",
        resources = "list",
        config = "list",
        rule = "character",
        bench_iteration = "numeric",
        scriptdir = "character",
        source = "function"
    )
)
snakemake <- Snakemake(
    input = list('analysis_large/rez/pangolin_lineage_report.csv', '/data/analyses/gisaid/metadata.csv', "pangolin" = 'analysis_large/rez/pangolin_lineage_report.csv', "meta" = '/data/analyses/gisaid/metadata.csv'),
    output = list('analysis_large/rez/most_abundand_lineages.csv', "stabilities_pairwise" = 'analysis_large/rez/most_abundand_lineages.csv'),
    params = list(),
    wildcards = list(),
    threads = 1,
    log = list('analysis_large/rez/most_abundand_lineages.ipynb', "notebook" = 'analysis_large/rez/most_abundand_lineages.ipynb'),
    resources = list('tmpdir', "tmpdir" = '/tmp'),
    config = list("work_dir" = 'analysis_large', "sequences" = '/data/analyses/gisaid/sequences.fasta', "meta" = '/data/analyses/gisaid/metadata.csv', "datasets_dir" = 'datasets', "lineages" = 'AY.4.5\tQ.1\tB.1.1.7\tB.1.177.60\tBA.2\tBA.2.9', "reference_id" = 'MN908947 (Wuhan-Hu-1/2019)', "reference_lineage" = 'BA.2', "cleanup_keep_fraction_of_columns" = 0.8, "cleanup_remove_proc_mostN_having" = 20, "target_amount" = 12000, "subset_amount_maximum" = 30, "maximum_amount_4_splits" = 2000, "reference" = 'datasets/NC_045512.gb'),
    rule = 'most_abundand_lineages',
    bench_iteration = as.numeric(NA),
    scriptdir = '/data/analyses/covid-phylogeny/snakefiles/notebooks',
    source = function(...){
        wd <- getwd()
        setwd(snakemake@scriptdir)
        source(...)
        setwd(wd)
    }
)
setwd('/data/analyses/covid-phylogeny');

######## snakemake preamble end #########


In [ ]:
# start coding here